In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import faiss
from sentence_transformers import util

/Users/valeriaalesnikova/Desktop/bootcamp/ghjtrn/smart_book_search/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Загрузка модели и токенизатора BERT
model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [3]:
books = pd.read_csv('/Users/valeriaalesnikova/Desktop/bootcamp/ghjtrn/smart_book_search/data/data_cleaned.csv')
books

,Unnamed: 0,page_url,image_url,author,title,annotation,cleaned_text,lemmatized_text,cleaned_title,text_length
0,0,https://www.biblio-globus.ru//product/10957168,https://static1.bgshop.ru/imagehandler.ashx?fi...,Маринина А.,Ричард Третий и Генрих Восьмой глазами Шекспира,Расследование жизни и деяний двух самых опороч...,расследование жизни деяний самых опороченных в...,расследование жизнь деяние самый опорочить воз...,ричард третий и генрих восьмой глазами ше...,1046
1,1,https://www.biblio-globus.ru//product/10997075,https://static1.bgshop.ru/imagehandler.ashx?fi...,Михалкова Е.И.,Знак истинного пути,Сыщики Макар Илюшин и Сергей Бабкин расследуют...,сыщики макар илюшин сергей бабкин расследуют п...,сыщик макар илюшин сергей бабкин расследовать ...,знак истинного пути,242
2,2,https://www.biblio-globus.ru//product/10941892,https://static1.bgshop.ru/imagehandler.ashx?fi...,Маринина А.,Генрих Шестой глазами Шекспира,Генрих Шестой – самая трагическая фигура на ан...,генрих самая трагическая фигура английском пре...,генрих самый трагический фигура английский пре...,генрих шестой глазами шекспира,1005
3,3,https://www.biblio-globus.ru//product/10978116,https://static1.bgshop.ru/imagehandler.ashx?fi...,Маринина А.,"Иоанн Безземельный, Эдуард Третий и Ричард Вто...",Истории трех знаменитых королей под одной обло...,истории трех знаменитых королей обложкой посво...,история три знаменитый король обложка посвоему...,иоанн безземельный эдуард третий и ричар...,1050
4,4,https://www.biblio-globus.ru//product/10845007,https://static1.bgshop.ru/extraimagehandler.as...,Маринина А.,Шпаргалка для ленивых любителей истории. Корол...,Первая книга в необычном для Александры Марини...,первая необычном александры марининой жанре по...,первый необычный александра маринин жанр попул...,шпаргалка для ленивых любителей истории. кор...,598
...,...,...,...,...,...,...,...,...,...,...
4747,5194,http://loveread.ec/view_global.php?id=16434,http://loveread.ec/img/photo_books/16434.jpg,Елена Чалова,Бизнес-план для аспирантки,Ирма из повести «Бизнес-план для аспирантки» и...,ирма повести бизнесплан аспирантки помышляла б...,ирма повесть бизнесплан аспирантка помышлять б...,бизнес-план для аспирантки,267
4748,5195,http://loveread.ec/view_global.php?id=16445,http://loveread.ec/img/photo_books/16445.jpg,Виталий Батюк,Глаза Вселенной видят все…,Основная идея романа – это давление обстоятель...,основная идея романа давление обстоятельств жи...,основный идея роман давление обстоятельство жи...,глаза вселенной видят все…,438
4749,5196,http://loveread.ec/view_global.php?id=16452,http://loveread.ec/img/photo_books/16452.jpg,Лаура Дэниелз,Для единственного…,Небольшой горный пансионат «Игл нест» в турист...,небольшой горный пансионат игл нест туристичес...,небольшой горный пансионат игла несть туристич...,для единственного…,255
4750,5197,http://loveread.ec/view_global.php?id=16461,http://loveread.ec/img/photo_books/16461.jpg,Лаура Дэниелз,Ключ к сердцу,"Пола привыкла думать, что все разговоры о любв...",пола привыкла разговоры любви духовном физичес...,пол привыкнуть разговор любовь духовный физиче...,ключ к сердцу,352


In [4]:
# Максимальная длина текста
MAX_LEN = 300

In [5]:
# Функция для встраивания текста с использованием BERT
def embed_bert_cls(text, model=model, tokenizer=tokenizer):
    t = tokenizer(text,
                  padding=True,
                  truncation=True,
                  return_tensors='pt',
                  max_length=MAX_LEN)
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().squeeze()

In [6]:
# Функция для обработки текста, чтобы все значения были строками
def preprocess_text(text):
    if isinstance(text, str):
        return text
    elif pd.isna(text):
        return ""
    else:
        return str(text)

# Применение функции к колонке 'annotation'
books['lemmatized_text'] = books['lemmatized_text'].apply(preprocess_text)

# Вычисление векторов для всех текстов в DataFrame
embeddings = []
for text in books['lemmatized_text']:  # Предполагаем, что тексты находятся в колонке 'annotation'
    embedding = embed_bert_cls(text)
    embeddings.append(embedding.numpy())

# Преобразование списка векторов в массив numpy
embeddings_array = np.stack(embeddings)

# Сохранение векторов в текстовый файл
np.savetxt('embeddings.txt', embeddings_array)

In [10]:
# Загрузка предварительно вычисленных векторов
embeddings = np.loadtxt('embeddings.txt')
embeddings_tensor = [torch.tensor(embedding) for embedding in embeddings]

# Создание индекса Faiss
embeddings_matrix = np.stack(embeddings)
index = faiss.IndexFlatIP(embeddings_matrix.shape[1])
index.add(embeddings_matrix)

# Текст запроса
text = 'Между сыщиком и преступником начинается жесткая игра'

# Встраивание запроса и поиск ближайших векторов с использованием Faiss
query_embedding = embed_bert_cls(text)
query_embedding = query_embedding.numpy().astype('float32')
k, indices = index.search(np.expand_dims(query_embedding, axis=0), 5)

# Вывод результатов
for i in indices[0]:
    print(books['title'][i])


Венчание со страхом
Тайна магазина игрушек
Фартовый город
Помеченный смертью
Ловцы черных душ. Дело глазника
